In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import json
import requests
import openpyxl
import urllib
from urllib import parse
import lxml
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from openpyxl import Workbook
from bs4 import BeautifulSoup
from openpyxl.utils.dataframe import dataframe_to_rows
from tensorflow.keras import layers
from nltk.corpus import stopwords
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import webdriver_manager
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

#웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

In [34]:
def l2Norm(inp):# 환율데이터 표본적어서 *10
    result = []
    for i in inp:
        result.append(((i-np.mean(inp))/np.var(inp))*10)
    return result
def l3Norm(inp):# *10안함
    result = []
    for i in inp:
        result.append(((i-np.mean(inp))/np.var(inp)))
    return result

class Stock:
    
    Exchange='./Data/Exchange.csv' #객체가 공통으로 사용할 환율 데이터 (동적크롤링할줄몰라서 csv로 받아옴)
    
    def __init__(self,code):
        self.code=code
    
    @staticmethod
    def exchangeRate():  ##1달간격의 환율의 종가데이터  1행배열 57요소( 18.05.01~  23.01.01) 배열로 출력 SIZE(1,57)
        csv = pd.read_csv(Stock.Exchange,encoding='utf8')
        #print(csv)  환율 전체데이터 확인하기            
        csv = np.array(csv)
        data=[]
        for i in range(len(csv)):   
            fnum=""
            for j in csv[i][1]:
                if(j == ','):
                    continue
                fnum+=j
            data.append(float(fnum))
        data = np.array(data)
        data = data.astype('float32')
        data = l2Norm(data) 
        return data


    def idxNorm(self):##ROA, ROE, EPS, BPS, DPS, PER, PBR의 각 항목당 (2018.12 ~ 2023.12)의 데이터 6개 갖음 SIZE(7,8)
        get_param = {
            'pGB':1,
            'gicode':'A%s'%(self.code),
            'cID':'',
            'MenuYn':'Y',
            'ReportGB':'',
            'NewMenuID':101,
            'stkGb':701,
        }
        get_param = parse.urlencode(get_param)
        url="http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?%s"%(get_param)
        tables = pd.read_html(url, header=0,encoding='utf-8')
        sit=np.array(tables[11]) 
        data=[]#정규화 시작
        idx=np.array([])
        for i in range(7):
            idx = sit[17+i][2:8].astype('float32')
            idx =l3Norm(idx)
            data.append(idx)
        data=np.array(data)
        return data
    
    def priceList(self):
        csv = pd.read_csv('./Data/주식가격/%s.csv'%(self.code))
        cut_csv = csv[:][:1148]
        array = np.array(cut_csv)
        close = array[:,2].reshape(-1,1)
        
        return close

    def showChart(self): ##종목코드의 제무재표를 나타내는 함수
        get_param = {
            'pGB':1,
            'gicode':'A%s'%(self.code),
            'cID':'',
            'MenuYn':'Y',
            'ReportGB':'',
            'NewMenuID':101,
            'stkGb':701,
        }
        get_param = parse.urlencode(get_param)
        url="http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?%s"%(get_param)
        tables = pd.read_html(url, header=0,encoding='utf-8')
        return tables[11]

In [35]:
SS = Stock('005930')
SK = Stock('000660')
DW=Stock('049770')
SFA=Stock('056190')
GW=Stock('001570')

In [85]:
sa=SS.priceList()
sk=SK.priceList()
dw=DW.priceList()
sf=SFA.priceList()
gy=GW.priceList()
sadata=SS.idxNorm()
skdata=SK.idxNorm()
dwdata=DW.idxNorm()
gydata=GW.idxNorm()
sfdata=SFA.idxNorm()
print(sadata.shape)

C:\Users\wjddu\AppData\Local\Temp\ipykernel_17884\1384323847.py:62: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  csv = pd.read_csv('./Data/주식가격/%s.csv'%(self.code))
C:\Users\wjddu\AppData\Local\Temp\ipykernel_17884\1384323847.py:62: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  csv = pd.read_csv('./Data/주식가격/%s.csv'%(self.code))


(7, 6)


In [86]:
sa2023 = sa[:-1147]
sa2022 = sa[-1147:-901]
sa2021 = sa[-901:-653]
sa2020 = sa[-653:-405]
sa2019 = sa[-405:-159]
sa2018 = sa[-159:len(sa)]

sk2023 = sk[:-1147]
sk2022 = sk[-1147:-901]
sk2021 = sk[-901:-653]
sk2020 = sk[-653:-405]
sk2019 = sk[-405:-159]
sk2018 = sk[-159:len(sa)]

dw2023 = dw[:-1147]
dw2022 = dw[-1147:-901]
dw2021 = dw[-901:-653]
dw2020 = dw[-653:-405]
dw2019 = dw[-405:-159]
dw2018 = dw[-159:len(sa)]

gy2023 = gy[:-1147]
gy2022 = gy[-1147:-901]
gy2021 = gy[-901:-653]
gy2020 = gy[-653:-405]
gy2019 = gy[-405:-159]
gy2018 = gy[-159:len(sa)]

sf2023 = sf[:-1147]
sf2022 = sf[-1147:-901]
sf2021 = sf[-901:-653]
sf2020 = sf[-653:-405]
sf2019 = sf[-405:-159]
sf2018 = sf[-159:len(sa)]

sadata2023 = np.full((len(sa2023),7), sadata[:,5])
sadata2022 = np.full((len(sa2022),7), sadata[:,4])
sadata2021 = np.full((len(sa2021),7), sadata[:,3])
sadata2020 = np.full((len(sa2020),7), sadata[:,2])
sadata2019 = np.full((len(sa2019),7), sadata[:,1])
sadata2018 = np.full((len(sa2018),7), sadata[:,0])

skdata2023 = np.full((len(sa2023),7), skdata[:,5])
skdata2022 = np.full((len(sa2022),7), skdata[:,4])
skdata2021 = np.full((len(sa2021),7), skdata[:,3])
skdata2020 = np.full((len(sa2020),7), skdata[:,2])
skdata2019 = np.full((len(sa2019),7), skdata[:,1])
skdata2018 = np.full((len(sa2018),7), skdata[:,0])

dwdata2023 = np.full((len(sa2023),7), dwdata[:,5])
dwdata2022 = np.full((len(sa2022),7), dwdata[:,4])
dwdata2021 = np.full((len(sa2021),7), dwdata[:,3])
dwdata2020 = np.full((len(sa2020),7), dwdata[:,2])
dwdata2019 = np.full((len(sa2019),7), dwdata[:,1])
dwdata2018 = np.full((len(sa2018),7), dwdata[:,0])

gydata2023 = np.full((len(sa2023),7), gydata[:,5])
gydata2022 = np.full((len(sa2022),7), gydata[:,4])
gydata2021 = np.full((len(sa2021),7), gydata[:,3])
gydata2020 = np.full((len(sa2020),7), gydata[:,2])
gydata2019 = np.full((len(sa2019),7), gydata[:,1])
gydata2018 = np.full((len(sa2018),7), gydata[:,0])

sfdata2023 = np.full((len(sa2023),7), sfdata[:,5])
sfdata2022 = np.full((len(sa2022),7), sfdata[:,4])
sfdata2021 = np.full((len(sa2021),7), sfdata[:,3])
sfdata2020 = np.full((len(sa2020),7), sfdata[:,2])
sfdata2019 = np.full((len(sa2019),7), sfdata[:,1])
sfdata2018 = np.full((len(sa2018),7), sfdata[:,0])

In [87]:
sadata1 = sadata2023
sadata1 = np.concatenate((sadata1,sadata2022,sadata2021,sadata2020,sadata2019,sadata2018))
sa = np.concatenate((sa, sadata1), axis=1)
skdata1 = skdata2023
skdata1 = np.concatenate((skdata1,skdata2022,skdata2021,skdata2020,skdata2019,skdata2018))
sk = np.concatenate((sk, skdata1), axis=1)
dwdata1 = dwdata2023
dwdata1 = np.concatenate((dwdata1,dwdata2022,dwdata2021,dwdata2020,dwdata2019,dwdata2018))
dw = np.concatenate((dw, dwdata1), axis=1)
gydata1 = gydata2023
gydata1 = np.concatenate((gydata1,gydata2022,gydata2021,gydata2020,gydata2019,gydata2018))
gy = np.concatenate((gy, gydata1), axis=1)
sfdata1 = sfdata2023
sfdata1 = np.concatenate((sfdata1,sfdata2022,sfdata2021,sfdata2020,sfdata2019,sfdata2018))
sf = np.concatenate((sf, sfdata1), axis=1)

In [88]:
sa = np.concatenate([sa, sk, dw, gy, sf], axis = 0)
array = np.array(sa)


In [97]:
er = Stock.exchangeRate()
er = np.array(er).reshape(-1, 1)
exchange = np.full((1,1), er[-57])
exchange = np.concatenate((exchange, np.full((21,1), er[-56]),
                           np.full((22,1), er[-55]),
                           np.full((19,1), er[-54]),
                           np.full((20,1), er[-53]),
                           np.full((22,1), er[-52]),
                           np.full((21,1), er[-51]),
                           np.full((20,1), er[-50]),
                           np.full((21,1), er[-49]),
                           np.full((21,1), er[-48]),
                           np.full((21,1), er[-47]),
                           np.full((18,1), er[-46]),
                           np.full((20,1), er[-45]),
                           np.full((22,1), er[-44]),
                           np.full((22,1), er[-43]),
                           np.full((19,1), er[-42]),
                           np.full((19,1), er[-41]),
                           np.full((21,1), er[-40]),
                           np.full((22,1), er[-39]),
                           np.full((22,1), er[-38]),
                           np.full((19,1), er[-37]),
                           np.full((22,1), er[-36]),
                           np.full((22,1), er[-35]),
                           np.full((18,1), er[-34]),
                           np.full((20,1), er[-33]),
                           np.full((21,1), er[-32]),
                           np.full((21,1), er[-31]),
                           np.full((19,1), er[-30]),
                           np.full((21,1), er[-29]),
                           np.full((20,1), er[-28]),
                           np.full((23,1), er[-27]),
                           np.full((22,1), er[-26]),
                           np.full((19,1), er[-25]),
                           np.full((20,1), er[-24]),
                           np.full((22,1), er[-23]),
                           np.full((20,1), er[-22]),
                           np.full((20,1), er[-21]),
                           np.full((20,1), er[-20]),
                           np.full((21,1), er[-19]),
                           np.full((21,1), er[-18]),
                           np.full((19,1), er[-17]),
                           np.full((21,1), er[-16]),
                           np.full((23,1), er[-15]),
                           np.full((19,1), er[-14]),
                           np.full((21,1), er[-13]),
                           np.full((22,1), er[-12]),
                           np.full((20,1), er[-11]),
                           np.full((17,1), er[-10]),
                           np.full((22,1), er[-9]),
                           np.full((19,1), er[-8]),
                           np.full((22,1), er[-7]),
                           np.full((21,1), er[-6]),
                           np.full((17,1), er[-5]),
                           np.full((22,1), er[-4]),
                           np.full((22,1), er[-3]),
                           np.full((19,1), er[-2]),
                           np.full((17,1), er[-1])), axis=0)
exchange_inputs = np.concatenate((exchange, exchange, 
                           exchange, exchange, exchange),axis=0)

In [98]:
train_inputs = np.concatenate((array, exchange_inputs), axis=1)
print(train_inputs.shape)

(5740, 9)


In [99]:
input_size = 9
model = Sequential([
    layers.LSTM(input_size,activation='tanh',return_sequences=True,input_shape=(input_size,1)),
    layers.LSTM(64,return_sequences=False),
    layers.Dense(20, activation='softmax')
])
model.compile(loss='mse',optimizer='adam')
model.summary()
model.fit(
    train_inputs,
    train_labels,
    validation_data=(x_test,y_test),
    batch_size=10,
    epochs=20
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 9, 9)              396       
                                                                 
 lstm_1 (LSTM)               (None, 64)                18944     
                                                                 
 dense (Dense)               (None, 20)                1300      
                                                                 
Total params: 20,640
Trainable params: 20,640
Non-trainable params: 0
_________________________________________________________________


NameError: name 'x_train' is not defined

In [ ]:
pred = model.predict(x_test)
fit = plt.figure(facecolor='white')
ax = fig.add_subplot(111)
ax.plot(y_test,label='True')
ax.plot(pred, label='Prediction')
ax.legend()
plt.show()